In [2]:
from datetime import datetime
import pandas as pd
import streamlit as st
import requests
import trafilatura
import urllib3
urllib3.disable_warnings()

In [3]:
urls = {
    'internet':'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/retailindustry/datasets/retailsalesindexinternetsales/current/internetreferencetables.xlsx',
    'pounds':'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/retailindustry/datasets/poundsdatatotalretailsales/current/poundsdata.xlsx',
    'index':'https://www.ons.gov.uk/file?uri=/economy/inflationandpriceindices/datasets/consumerpriceindices/current/mm23.xlsx',
    'valvol':'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/retailindustry/datasets/retailsalesindexreferencetables/current/mainreferencetables.xlsx'
}

AVERAGE_WEEKLY = 52/12

# download each workbook and save as file
for _ in urls:
    with open(f'./.data/input/{_}.xlsx', 'wb') as output:
        output.write(requests.get(urls[_],verify=False).content)

In [4]:
dfOnline = pd.ExcelFile('./.data/input/internet.xlsx').parse('IntValSA')

# column fix
dfOnline.columns = dfOnline.loc[2]
dfOnline.columns.name = None
dfOnline.columns = dfOnline.columns.str.replace(r'\[Note \d+\]|\[note \d+\]',
                                                '',
                                                regex=True)
dfOnline.columns = dfOnline.columns.str.strip()
dfOnline.rename(columns={"All retailing excluding automotive fuel": "Total online (£bn, avg. weekly)"}, inplace=True)

# index fix
dfOnline['Time Period'] = pd.to_datetime(dfOnline['Time Period'],
                                         format='%Y %b',
                                         errors='coerce').dt.date
dfOnline = dfOnline[dfOnline['Time Period'] >= pd.to_datetime('2020-01-01').date()]
dfOnline = dfOnline.set_index('Time Period')

# reformat
dfOnline = (dfOnline / 1000
            # * AVERAGE_WEEKLY
            ).astype(float).round(3)
dfOnline['Non-food and other stores'] = dfOnline['Total of predominantly non-food stores'] + dfOnline['Non-store retailing']

# yoy fix
dfOnlineYoy = (dfOnline.pct_change(periods=12).dropna() * 100).round(1)
dfOnlineYoy.rename(columns={"Total online (£bn, avg. weekly)": "Total online (excl. Fuel)"}, inplace=True)

# final dataframe
dfOnlineProc = pd.concat([dfOnline['Total online (£bn, avg. weekly)'],
                    dfOnlineYoy['Total online (excl. Fuel)'],
                    dfOnlineYoy['Predominantly food stores'],
                    dfOnlineYoy['Non-food and other stores']
                    ], axis=1)

dfOnlineProc.index.name = None

dfOnlineProc.tail()

,"Total online (£bn, avg. weekly)",Total online (excl. Fuel),Predominantly food stores,Non-food and other stores
2023-06-01,2.392,8.0,9.6,7.8
2023-07-01,2.452,10.0,7.9,10.4
2023-08-01,2.444,9.8,11.4,9.5
2023-09-01,2.410,7.2,12.3,6.3
2023-10-01,2.382,5.1,4.7,5.2


In [5]:
dfPounds = pd.ExcelFile('./.data/input/pounds.xlsx').parse('ValSAW')

# column fix
dfPounds.columns = dfPounds.loc[3]
dfPounds.columns.name = None
dfPounds.columns = dfPounds.columns.str.strip()
dfPounds.rename(columns={"All Retailing excluding automotive fuel": "Total sales (£bn, avg. weekly)"}, inplace=True)
dfPounds = dfPounds.drop(columns=['Number of Weeks',
                                  'All retailing including automotive fuel',
                                  'Month as a % of Total',
                                  'Month as a % of Total excluding automotive fuel',
                                  'Total Annual Sales for All Retailing including automotive fuel',
                                  'Total Annual Sales for All Retailing excluding automotive fuel',
                                  'Automotive fuel'])

# index fix
dfPounds['Time Period'] = pd.to_datetime(dfPounds['Time Period'],
                                         format='%Y%b',
                                         errors='coerce').dt.date
dfPounds = dfPounds[dfPounds['Time Period'] >= pd.to_datetime('2020-01-01').date()]
dfPounds = dfPounds.set_index('Time Period')

# reformat
dfPounds = ((dfPounds / 1000000
             ) 
            # * AVERAGE_WEEKLY
            ).astype(float).round(3)
dfPounds['Non-food and other stores'] = dfPounds['Total non-food stores'] + dfPounds['Non-store retailing']

# yoy fix
dfPoundsYoy = (dfPounds.pct_change(periods=12).dropna() * 100).round(1)
dfPoundsYoy.rename(columns={"Total sales (£bn, avg. weekly)": "Total sales (excl. Fuel)"}, inplace=True)

# final dataframe
dfPoundsProc = pd.concat([dfPounds['Total sales (£bn, avg. weekly)'],
                    dfPoundsYoy['Total sales (excl. Fuel)'],
                    dfPoundsYoy['Predominantly food stores'],
                    dfPoundsYoy['Non-food and other stores']
                    ], axis=1)

dfPoundsProc.index.name = None

In [7]:
dfIndex = pd.read_excel('./.data/input/index.xlsx') # read time is approx 2m

c:\Users\19732\.virtualenvs\ons-retail-sales-7xXVvHec\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
dfIndexProc = dfIndex.filter(regex='CPI ANNUAL RATE')
dfIndexProc.insert(0, 'Time Period', dfIndex.iloc[:, 0])
dfIndexProc = dfIndexProc.set_index('Time Period')
dfIndexProc = dfIndexProc.loc['2020 JAN':]
dfIndexProc.index = pd.to_datetime(dfIndexProc.index,
                                   format='%Y %b',
                                   errors='coerce').date

##################### comment out for all annual rates #######################
keepColumns = ['CPI ANNUAL RATE 00: ALL ITEMS 2015=100',
'CPI ANNUAL RATE 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100',
'CPI ANNUAL RATE 02 : ALCOHOLIC BEVERAGES,TOBACCO 2015=100',
'CPI ANNUAL RATE 03 : CLOTHING AND FOOTWEAR 2015=100',
'CPI ANNUAL RATE 05 : FURN, HH EQUIP & REPAIR OF THE HOUSE 2015=100',
'CPI ANNUAL RATE 08 : COMMUNICATION 2015=100',
'CPI ANNUAL RATE 09 : RECREATION & CULTURE 2015=100',
'CPI ANNUAL RATE 11 : HOTELS, CAFES AND RESTAURANTS 2015=100',
]
dfIndexProc = dfIndexProc[keepColumns]
##############################################################################
dfIndexProc = dfIndexProc.iloc[:,:40]
dfIndexProc.columns = dfIndexProc.columns.to_series().replace({
    r'CPI ANNUAL RATE \d{2}(\.\d+)?': '',
    r'2015=100': '',
    r':': ''
}, regex=True)

dfIndexProc.rename(columns=lambda x: x.title(), inplace=True)
dfIndexProc.columns = dfIndexProc.columns.str.strip()

new_columns = [
    'CPI (YoY%)',
    'Food & Non-Alcoholic Drinks',
    'Alcoholic Drinks & Tobacco',
    'Clothing & Footwear',
    'Furniture & Household Equipment',
    'Communication',
    'Recreation (incl. Electronics)',
    'Hotels, Cafes & Restaurants',
]

dfIndexProc.rename(columns=dict(zip(dfIndexProc.columns, new_columns)), inplace=True)

In [9]:
dfValVol = pd.read_excel('./.data/input/valvol.xlsx', sheet_name=['CPSA', 'KPSA'])

def process_dataframe(df,columnName):
    # Set columns and remove notes
    df.columns = df.loc[5]
    df.columns.name = None
    df.columns = df.columns.str.replace(r'\[note\d+\]|\[Note \d+\]|\[note \d+\]', '', regex=True)

    # Convert 'Time Period' to datetime
    df['Time Period'] = pd.to_datetime(df['Time Period'],
                                       format='%Y %b',
                                       errors='coerce').dt.date
    # Slice and set index
    df = df.iloc[392:438, 0:3:2]
    df = df.set_index('Time Period')
    df.columns.values[0] = columnName

    return df

dfValVol['CPSA'] = process_dataframe(dfValVol['CPSA'], 'Retail sales value (Index)')
dfValVol['KPSA'] = process_dataframe(dfValVol['KPSA'], 'Retail sales volume (Index)')

dfValVolProc = pd.concat([dfIndexProc['CPI (YoY%)'], dfValVol['CPSA'], dfValVol['KPSA']], axis=1)

In [15]:
with pd.ExcelWriter('./final_proc.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet
    dfOnlineProc.to_excel(writer, sheet_name='Internet_Data')
    dfPoundsProc.to_excel(writer, sheet_name='Overall_Data')
    dfIndexProc.to_excel(writer, sheet_name='CPI_Data')
    dfValVolProc.to_excel(writer, sheet_name='Val-Vol_Divergence_Data')
    
with pd.ExcelWriter('./final_full.xlsx', engine='xlsxwriter') as writer:
    dfOnline.to_excel(writer, sheet_name='Internet_Data')
    dfPounds.to_excel(writer, sheet_name='Overall_Data')

# Text download

In [11]:
with open('./text_downloads/cpi/cpi_oct2023.txt', "w", encoding="utf-8") as file:
    file.write(trafilatura.extract(trafilatura.fetch_url('https://www.ons.gov.uk/economy/inflationandpriceindices/bulletins/consumerpriceinflation/november2023'), include_tables=False))

In [12]:
# datetime.now().strftime('%B').lower()
# (datetime.now().replace(day=1) - datetime.timedelta(days=1)).strftime('%B')

with open('./text_downloads/retail_sales/ons_text.txt', "w", encoding="utf-8") as file:
    file.write(trafilatura.extract(trafilatura.fetch_url('https://www.ons.gov.uk/businessindustryandtrade/retailindustry/bulletins/retailsales/november2023'), include_tables=False))

TypeError: write() argument must be str, not None